# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

# Índice <a name="Contents"></a>
1.   [Ler o arquivo original](#ler)
2.   [Verificar de duplicatas](#duplicatas)
3.   [Conhecer a estrutura da tabela](#estrutura)
4.   [Valores "missing" de cada coluna](#missing)
5.   [Reduzir a quantidade de colunas da base de dados](#reduzindo)
6.   [Remover os registros com 'APGAR5' não preenchido](#apgar5)
7.   [Tratar as colunas 'ESTCIVMAE' e 'CONSULTAS'](#sub9)
8.   [Tratar a coluna 'QTDFILVIVO'](#filhos)
9.   [Modificar o tipo de dados](#tipos)
10. [Tratar 'LOCNASC', 'IDADEMAE', 'ESCMAE', 'GESTACAO', 'GRAVIDEZ'](#outros)
11. [Classificar o 'APGAR5'](#class)
12. [Renomear variáveis](#renomear)

# 1. Ler o arquivo original <a name="ler"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [289]:
import pandas as pd
import requests

# Importando o arquivo 
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Analisando o "formato" do arquivo original
print(sinasc.shape)

(27028, 69)


# 2. Verificar das duplicatas <a name="duplicatas"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [290]:
# Exclusão das duplicatas
sinasc.drop_duplicates().shape

sinasc = pd.DataFrame(sinasc)

In [291]:
print(sinasc.shape)

(27028, 69)


Há a mesma quantidade de linhas antes e depois da exclusão de duplicatas, então, conclui-se que não há linhas duplicadas.

# 3. Conhecer a estrutura da tabela  <a name="estrutura"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [292]:
# Imprimindo parte dela
sinasc.head(5)

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [293]:
# Informações de todas as colunas
sinasc.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

# 4. Valores "missing" de cada coluna<a name="missing"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)


In [294]:
# Contagem de missing por coluna

for coluna in sinasc.columns:
    null_count = sinasc[coluna].isnull().sum()

    # Imprimindo apenas os dados das colunas com "missing"
    if null_count != 0:
        print(coluna,':',null_count)

CODESTAB : 115
ESTCIVMAE : 317
ESCMAE : 312
CODOCUPMAE : 2907
QTDFILVIVO : 1573
QTDFILMORT : 2098
GESTACAO : 1232
GRAVIDEZ : 79
PARTO : 49
HORANASC : 21
SEXO : 4
APGAR1 : 96
APGAR5 : 103
RACACOR : 647
IDANOMAL : 591
CODANOMAL : 26814
DTRECORIGA : 27028
NATURALMAE : 298
CODMUNNATU : 298
CODUFNATU : 298
ESCMAE2010 : 249
SERIESCMAE : 12710
DTNASCMAE : 40
RACACORMAE : 661
QTDGESTANT : 1212
QTDPARTNOR : 1879
QTDPARTCES : 1791
IDADEPAI : 19421
DTULTMENST : 10072
SEMAGESTAC : 1238
TPMETESTIM : 1238
CONSPRENAT : 930
MESPRENAT : 2867
TPAPRESENT : 265
STTRABPART : 947
STCESPARTO : 747
TPNASCASSI : 61
TPFUNCRESP : 67
TPDOCRESP : 14
DTDECLARAC : 52
ESCMAEAGR1 : 249
munResLat : 1
munResLon : 1
munResAlt : 1
munResArea : 1


In [295]:
# Verficando as colunas CODANOMAL e DTRECORIGA

sinasc[['CODANOMAL', 'DTRECORIGA']]

,CODANOMAL,DTRECORIGA
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
27023,NaN,NaN
27024,NaN,NaN
27025,NaN,NaN
27026,NaN,NaN


# 5. Reduzir a quantidade de colunas da base de dados <a name="reduzindo"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [296]:
# Redução das colunas 
# Usar colunas ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']


# Criando um dicionário com os dados das colunas de interesse
sinasc_dic = ({'LOCNASC': sinasc['LOCNASC'], 'IDADEMAE': sinasc['IDADEMAE'], 'ESTCIVMAE': sinasc['ESTCIVMAE'], 'ESCMAE': sinasc['ESCMAE'], 
               'QTDFILVIVO': sinasc['QTDFILVIVO'],   'GESTACAO': sinasc['GESTACAO'], 'GRAVIDEZ': sinasc['GRAVIDEZ'], 
               'CONSULTAS': sinasc['CONSULTAS'], 'APGAR5': sinasc['APGAR5']})

#Criando o novo DataFrame 
sinasc = pd.DataFrame(sinasc_dic)

sinasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [297]:
# Contando o número valores missing por coluna no novo DataFrame
sinasc.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

# 6. Removendo os registros com 'APGAR5' não preenchido <a name="apgar5"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [298]:
# Remover as 103 linhas em que o valor de Apgar5 é null

sinasc = sinasc.dropna(subset = 'APGAR5')
sinasc.shape

(26925, 9)

In [299]:
# Contando o número valores missing por coluna no novo DataFrame
sinasc.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [300]:
# Consultando os possíveis dados da coluna APGAR5
print(sinasc['APGAR5'].unique())

[10.  9.  7.  0.  8.  4.  3.  6.  5.  2.  1.]


# 7. Tratado as colunas 'ESTCIVMAE' e 'CONSULTAS' <a name="sub9"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [301]:
# Tipo de dados de cada coluna
sinasc.dtypes

LOCNASC         int64
IDADEMAE        int64
ESTCIVMAE     float64
ESCMAE         object
QTDFILVIVO    float64
GESTACAO       object
GRAVIDEZ       object
CONSULTAS       int64
APGAR5        float64
dtype: object

In [302]:
# Substituindo NaN por 9 nas 315 linhas de ESTCIVMAE, tipo float, com valor missing, 
# notando que não há linhas com valor missing em CONSULTAS

# Copiando o DataFrame para evitar o posterior erro "A value is trying to be set on a copy of a slice from a DataFrame"
sinasc = sinasc.copy()

#Substituindo
sinasc['ESTCIVMAE'].fillna(9.0, inplace = True)

#Consultar a quantidade de missing na coluna ESTCIVMAE
print(sinasc['ESTCIVMAE'].isnull().sum())

#Consultar os dados possíveis na ESTCIVMAE
print(sinasc['ESTCIVMAE'].unique())

0
[5. 2. 9. 1. 4. 3.]


In [303]:
# Consultando os possíveis valores em CONSULTAS
print('São possíveis', sinasc['CONSULTAS'].nunique(),'valores, que são:')
print(sinasc['CONSULTAS'].unique())

São possíveis 5 valores, que são:
[4 3 2 1 9]


# 8. Tratar a coluna 'QTDFILVIVO' <a name="filhos"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)


In [304]:
#Substituindo as 1566 linhas missing da coluna 'QTDFILVIVO' , tipo float, por 0
sinasc['QTDFILVIVO'].fillna(0., inplace = True )

#Consultar a quantidade de missing na coluna QTDFILVIVO
print(sinasc['QTDFILVIVO'].isnull().sum())

#Consultar os dados possíveis na QTDFILVIVO
print(sinasc['QTDFILVIVO'].unique())

0
[ 0.  1.  2.  3.  4.  6.  5.  7. 12.  9. 11.  8. 30. 10. 14.]


# 9. Modificar o tipo de dados  <a name="tipos"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

Os valores das colunas ESTCIVMAE, QTDFILVIVO e APGAR5 não são possíveis de ser decimais e estão descritos tipo float, assim modificarei os dados dessas colunas para o tipo inteiro.

In [305]:
# Substituindo as colunas float por int
sinasc['ESTCIVMAE'] = sinasc['ESTCIVMAE'].astype(int)
sinasc['QTDFILVIVO'] = sinasc['QTDFILVIVO'].astype(int)
sinasc['APGAR5'] = sinasc['APGAR5'].astype(int)
sinasc.dtypes

LOCNASC        int64
IDADEMAE       int64
ESTCIVMAE      int32
ESCMAE        object
QTDFILVIVO     int32
GESTACAO      object
GRAVIDEZ      object
CONSULTAS      int64
APGAR5         int32
dtype: object

# 10. Tratar 'LOCNASC', 'IDADEMAE', 'ESCMAE', 'GESTACAO', 'GRAVIDEZ'  <a name="outros"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)


In [306]:
# Relação coluna x dados missing, mais uma vez
sinasc.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [307]:
# Ainda devem ser tratadas devido a "missing" dados 'ESCMAE', 'GESTACAO', 'GRAVIDEZ' , ambas tipo object
#Verificando as possíveis respostas para cada 

print('ESCMAE: São possíveis', sinasc['ESCMAE'].nunique(),'valores, que são:')
print(sinasc['ESCMAE'].unique(),'\n')

print('GESTACAO: São possíveis', sinasc['GESTACAO'].nunique(),'valores, que são:')
print(sinasc['GESTACAO'].unique(),'\n')

print('GRAVIDEZ: São possíveis', sinasc['GRAVIDEZ'].nunique(),'valores, que são:')
print(sinasc['GRAVIDEZ'].unique(),'\n')

ESCMAE: São possíveis 5 valores, que são:
['8 a 11 anos' '12 anos ou mais' '1 a 3 anos' '4 a 7 anos' nan 'Nenhuma'] 

GESTACAO: São possíveis 6 valores, que são:
['37 a 41 semanas' '42 semanas e mais' '32 a 36 semanas' '22 a 27 semanas'
 '28 a 31 semanas' nan 'Menos de 22 semanas'] 

GRAVIDEZ: São possíveis 3 valores, que são:
['Única' 'Dupla' nan 'Tríplice e mais'] 



Pela estrutura do banco de dados sabemos que em todas as colunas observadas é "permitido" o uso do valor Ignorado em substituição ao nan apresentado. 

In [308]:
#Substituindo missing em cada uma das colunas posteriores
sinasc['ESCMAE'].fillna('Ignorado', inplace = True )
sinasc['GESTACAO'].fillna('Ignorado', inplace = True )
sinasc['GRAVIDEZ'].fillna('Ignorado', inplace = True )

# Confimando substituição
print('ESCMAE:',sinasc['ESCMAE'].isnull().sum(), 'valores únicos')
print('GESTACAO:',sinasc['GESTACAO'].isnull().sum(), 'valores únicos')
print('GRAVIDEZ:',sinasc['GRAVIDEZ'].isnull().sum(), 'valores únicos')

ESCMAE: 0 valores únicos
GESTACAO: 0 valores únicos
GRAVIDEZ: 0 valores únicos


In [309]:
#Verificando os dados em 'LOCNASC', 'IDADEMAE'

print('LOCNASC: São possíveis', sinasc['LOCNASC'].nunique(),'valores, que são:')
print(sinasc['LOCNASC'].unique(),'\n')

print('IDADEMAE: São possíveis', sinasc['IDADEMAE'].nunique(),'valores, que são:')
print(sinasc['IDADEMAE'].unique(),'\n')

LOCNASC: São possíveis 4 valores, que são:
[1 4 2 3] 

IDADEMAE: São possíveis 40 valores, que são:
[19 29 37 30 33 27 20 31 21 18 24 22 25 26 34 23 28 14 17 40 16 35 39 38
 32 36 41 15 42 43 12 13 47 45 44 11 53 46 51 52] 



Nenhuma das duas colunas aparentam inconsistência de valores

# 11. Classificar o 'APGAR5'  <a name="class"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

Dados de classificação do APGAR5:

        Entre 8 e 10 está em uma faixa 'normal'.
        Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
        Entre 4 e 5 significa 'asfixia moderada'.
        Entre 0 e 3 significa 'asfixia severa'.

In [310]:
# Consultando os possíveis dados da coluna APGAR5
print(sinasc['APGAR5'].unique())

[10  9  7  0  8  4  3  6  5  2  1]


In [311]:
# Criando a classificação na coluna CLASSIFICACAO APGAR5
sinasc.loc[sinasc['APGAR5'] <=3, 'CLASSIFICACAOAPGAR5'] = 'Asfixia Severa'
sinasc.loc[(sinasc['APGAR5'] >=4) & (sinasc['APGAR5'] <=5), 'CLASSIFICACAOAPGAR5'] = 'Asfixia Moderada'
sinasc.loc[(sinasc['APGAR5'] >=6) & (sinasc['APGAR5'] <=7), 'CLASSIFICACAOAPGAR5'] = 'Asfixia Leve'
sinasc.loc[sinasc['APGAR5'] >=8, 'CLASSIFICACAOAPGAR5'] = 'Normal'

sinasc.head(7)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,CLASSIFICACAOAPGAR5
0,1,19,5,8 a 11 anos,0,37 a 41 semanas,Única,4,10,Normal
1,1,29,2,8 a 11 anos,1,37 a 41 semanas,Única,4,9,Normal
2,1,37,9,8 a 11 anos,2,37 a 41 semanas,Única,4,10,Normal
3,1,30,5,12 anos ou mais,0,37 a 41 semanas,Única,3,10,Normal
4,1,30,2,8 a 11 anos,1,37 a 41 semanas,Única,4,10,Normal
5,1,33,5,1 a 3 anos,2,37 a 41 semanas,Única,4,10,Normal
6,1,37,1,8 a 11 anos,1,37 a 41 semanas,Única,4,10,Normal


In [312]:
# Frequências dessa categorização

# Contar o número de repetição de cada caso de 'CLASSIFICACAOAPGAR5'
sinasc['CLASSIFICACAOAPGAR5'].value_counts()

CLASSIFICACAOAPGAR5
Normal              26463
Asfixia Leve          320
Asfixia Severa         74
Asfixia Moderada       68
Name: count, dtype: int64

In [313]:
# % de cada CLASSIFICACAOAPGAR5

#Total de partos
total = sinasc.shape[0]

#%
aparicoes = round(100*sinasc['CLASSIFICACAOAPGAR5'].value_counts()/total, 2)

percentual = aparicoes.apply(lambda x: f"{x}%")
print(percentual)

CLASSIFICACAOAPGAR5
Normal              98.28%
Asfixia Leve         1.19%
Asfixia Severa       0.27%
Asfixia Moderada     0.25%
Name: count, dtype: object


# 12. Renomear variáveis  <a name="renomear"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [314]:
# Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. 

var_ori= []

# Reobservando o nome das variáveis
var_ori = sinasc.columns
var_ori = var_ori.astype(str)
var_ori

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'CLASSIFICACAOAPGAR5'],
      dtype='object')

In [315]:
#Convertendo para minúsculo 
var_min = [var.lower() for var in var_ori]
var_min

['locnasc',
 'idademae',
 'estcivmae',
 'escmae',
 'qtdfilvivo',
 'gestacao',
 'gravidez',
 'consultas',
 'apgar5',
 'classificacaoapgar5']

In [316]:
#Substituind o nome das variáveis no DF
sinasc.columns = ['loc_nasc',
 'idade_mae',
 'est_civ_mae',
 'esc_mae',
 'qtd_fil_vivo',
 'gestacao',
 'gravidez',
 'consultas',
 'apgar5',
 'class_apgar5']

In [317]:
sinasc

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,class_apgar5
0,1,19,5,8 a 11 anos,0,37 a 41 semanas,Única,4,10,Normal
1,1,29,2,8 a 11 anos,1,37 a 41 semanas,Única,4,9,Normal
2,1,37,9,8 a 11 anos,2,37 a 41 semanas,Única,4,10,Normal
3,1,30,5,12 anos ou mais,0,37 a 41 semanas,Única,3,10,Normal
4,1,30,2,8 a 11 anos,1,37 a 41 semanas,Única,4,10,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2,12 anos ou mais,1,32 a 36 semanas,Única,4,9,Normal
27024,1,19,1,8 a 11 anos,0,37 a 41 semanas,Única,4,9,Normal
27025,1,24,2,8 a 11 anos,0,37 a 41 semanas,Única,4,10,Normal
27026,1,21,2,8 a 11 anos,1,32 a 36 semanas,Única,4,9,Normal
